In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('../dataset/train.csv')

In [3]:
df.head(5)

,id,name,document_text,cat_name
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",Corporate Communications
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...",Securities Settlement
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,Antitrust
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,Securities Settlement
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,Financial Crime


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47102 entries, 0 to 47101
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             47102 non-null  int64 
 1   name           47102 non-null  object
 2   document_text  47102 non-null  object
 3   cat_name       47102 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [5]:
# there is no null value 
df.columns

Index(['id', 'name', 'document_text', 'cat_name'], dtype='object')

In [6]:
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
def convert_to_lower(df,col_name):
    df[col_name] = df[col_name].str.lower()
    return df

def tokenise(df,col_name):
    df[col_name] = df[col_name].apply(word_tokenize)
    return df

def remove_stopwords(df,col_name):
    stop_words = set(stopwords.words('english'))
    df[col_name] = df[col_name].apply(lambda x: [word for word in x if word not in stop_words])
    return df

def lemmatize(df,col_name):
    lemmatizer = WordNetLemmatizer()
    df[col_name] = df[col_name].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    return df

def remove_punctuation(df,col_name):
    df[col_name] = df[col_name].apply(lambda x: [word for word in x if word.isalpha()])
    return df

def join_words(df,col_name):
    df[col_name] = df[col_name].apply(lambda x: ' '.join(x))
    return df

def clean_text(df,col_name):
    df = convert_to_lower(df,col_name)
    df = tokenise(df,col_name)
    df = remove_stopwords(df,col_name)
    df = lemmatize(df,col_name)
    df = remove_punctuation(df,col_name)
    df = join_words(df,col_name)
    return df

# df = clean_text(df,'text')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
def remove_spaces(df,col_name):
    df[col_name] = df[col_name].str.strip()
    return df

df = remove_spaces(df,'cat_name')

In [25]:
#import countVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# import tfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline(
    [
        ('bow',CountVectorizer()),
        ('tfidf',TfidfTransformer()),
        ('classifier',MultinomialNB())
    ]
)

X = df.iloc[:,1:3]
y = df.iloc[:,3]
# # train test spllit
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
pipeline.fit(X_train,y_train)


ValueError: Found input variables with inconsistent numbers of samples: [2, 37681]

In [7]:
# we have to predict cat_name of document
df['cat_name'].value_counts()

Examinations                                1742
Securities Sales                            1737
Market Risk                                 1633
Regulatory Actions                          1621
Compliance Management                       1391
Information Filing                          1387
Legal Proceedings                           1343
Fees and Charges                            1301
Exemptions                                  1190
Financial Crime                             1178
Contract Provisions                         1153
Securities Clearing                         1141
Listing                                     1124
Securities Issuing                          1107
Payments and Settlements                    1099
Natural Disasters                           1079
Banking                                     1078
Regulatory Reporting                        1042
Licensing                                    999
Risk Management                              982
Licensure and certif